In [337]:
%logstart -o

import sys

import numpy as np
#in case we need to repeat experiment
np.random.seed(255)

import pandas as pd
pd.options.display.max_rows = 22

import matplotlib.pyplot as plt
plt.style.use('classic')


import seaborn as sns
sns.set()

#sklearn imports
from sklearn.cluster import KMeans
#from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV


#Use print instead of display when run as python script
pyscript = True

#Classifier verborsity where supported
verbose_level=1

#Sample Size
sampleN = 100000

import time

#Enable Optimization Algorithms
enable_grid_search = True
svm_c = 1
svm_gamma = 1
enable_rf_features = True
feature_all = False
defaultFeatures = ['P_AGE', 'V_YEAR', 'C_HOUR', 'C_YEAR', 'C_MNTH', 'C_CONF', 'C_WDAY', 'C_VEHS', 'P_USER', 'P_SEX']
enable_kmeans = True
enable_logistic_regression_l1 = True

# Enable Algorithms
enable_logistic_regression = True
enable_decision_tree = True
enable_svm = True
enable_knn = True
enable_mlp = True
enable_rf = True


C:\Users\pradeep\Anaconda3\lib\site-packages\IPython\core\magics\logging.py:130: UserWarning: Couldn't start log: Log file is already active: ipython_log.py
  warn("Couldn't start log: %s" % sys.exc_info()[1])


In [338]:
print("Sample size: {}".format(sampleN))

if enable_grid_search:
    print("Grid Search: Enabled")
else:
    print("Grid Search: Disabled")

if enable_rf_features:
    print("Feature Selection by RandomForest: Enabled")
else:
    print("Feature Selection by RandomForest: Disabled")
    print("Default Feature: {}".format(defaultFeatures))

if feature_all:
    print("All Features: Enabled")
else:
    print("All Features: Disabled")
    
if enable_kmeans:
    print("K-means: Enabled")
else:
    print("K-means: Disabled")

if enable_logistic_regression:
    print("Logistic Regression: Enabled")
else:
    print("Logistic Regression: Disabled")
    
if enable_decision_tree:
    print("Decision Tree: Enabled")
else:
    print("Decision Tree: Disabled")
    
if enable_svm:
    print("Support Vector Machines: Enabled")
else:
    print("Support Vector Machines: Disabled")

if  enable_knn:
    print("KNN: Enabled")
else:
    print("KNN: Disabled")
    
if enable_mlp:
    print("MLP: Enabled")
else:
    print("MLP: Disabled")
    
if enable_rf:
    print("Random Forest: Enabeld")
else:
    print("Random Forest: Disabled")

Sample size: 20
Grid Search: Enabled
Feature Selection by RandomForest: Disabled
Default Feature: ['P_AGE', 'V_YEAR', 'C_HOUR', 'C_YEAR', 'C_MNTH', 'C_CONF', 'C_WDAY', 'C_VEHS', 'P_USER', 'P_SEX']
K-means: Enabled
Logistic Regression: Enabled
Decision Tree: Enabled
Support Vector Machines: Enabled
KNN: Enabled
MLP: Enabled
Random Forest: Enabeld


In [339]:
#sys.stdout=open("Initial_Binary_Model2.out", "w")

In [340]:
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ))

Sun Nov 11 11:27:54 2018


In [341]:
#df = pd.read_csv('data01_simple.csv', engine = 'python')
df = pd.read_csv('data01_clean.csv', engine = 'python')
print(df.head(2))

   C_YEAR  C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  \
0    1999       1       1       9       2      34       2       1       1   
1    1999       1       1       9       2      34       2       1       1   

   C_RALN   ...    V_ID  V_TYPE  V_YEAR  P_ID  P_SEX  P_AGE  P_PSN  P_SAFE  \
0       1   ...       1       1    1992     1      0     33     11       2   
1       1   ...       2       1    1992     1      0     70     11       2   

   P_USER  P_ISEV  
0       1       2  
1       1       1  

[2 rows x 21 columns]


In [342]:
print(df.isnull().sum().sum())

0


In [343]:
print(df[df.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

0


In [344]:
df_cat = df.astype('category').copy()

In [345]:
df_int = df.astype('int').copy()

In [346]:
print(df.columns)

Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_ID', 'V_TYPE', 'V_YEAR',
       'P_ID', 'P_SEX', 'P_AGE', 'P_PSN', 'P_SAFE', 'P_USER', 'P_ISEV'],
      dtype='object')


## Convert Class Variable to Binary

In [347]:
## Convert Class Variable to Binary
### Merge Injury and Fatality as a single class
### we will compare the results.
df_binary_class = df_cat.copy()

#perform the conversion in two steps to avoid any unwanted side effects
df_binary_class['P_ISEV'] = df_binary_class['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
df_binary_class['P_ISEV'] = df_binary_class['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'1'})
print((df_binary_class['P_ISEV']=='0').sum())
print((df_binary_class['P_ISEV']=='1').sum())
print(df_binary_class['P_ISEV'].unique())

1570775
2084559
['1' '0']


In [348]:
df_100k = df_binary_class.sample(n=sampleN)

## Split Training and Testing for Binary class

In [349]:
#Split between data and class
Y = df_100k[df_binary_class.columns[-1]]
X = df_100k[df_binary_class.columns[0:df_binary_class.columns.size -1]]

#### Split Test(70%) and Train (30%) for Bianry class 

In [350]:
#sprint into train and test 70/30
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [351]:
#print(Xbinary_train, Xbinary_test, Ybinary_train, Ybinary_test)

## Write cleaned data to file for future use.

In [352]:
#lets write the datafile for future use
df_binary_class.to_csv('cleansimple_binary.csv', encoding='utf-8', index=False)

## Clustering based on K-Means Clustering

In [353]:
if enable_kmeans:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    print("K-Means Clustering: Start")
    kmeans = KMeans(n_clusters=3, init='random', n_init=10, max_iter=300, tol=1e-04, verbose= verbose_level)
    print("K-Means Clustering: Build")
    ykm = kmeans.fit(X)
    
    if pyscript:
        print(ykm.cluster_centers_)
        print(ykm.labels_)
    else:
        display(ykm.cluster_centers_)
        display(ykm.labels_)
    
    print("K-Means Clustering: End")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))


Sun Nov 11 11:30:33 2018
K-Means Clustering: Start
K-Means Clustering: Build
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 11181.05
start iteration
done sorting
end inner loop
Iteration 1, inertia 10713.125
start iteration
done sorting
end inner loop
Iteration 2, inertia 8859.333333333332
start iteration
done sorting
end inner loop
Iteration 3, inertia 7721.647435897436
start iteration
done sorting
end inner loop
Iteration 4, inertia 7721.647435897436
center shift 0.000000e+00 within tolerance 4.622175e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 10975.125
start iteration
done sorting
end inner loop
Iteration 1, inertia 10818.28968253968
start iteration
done sorting
end inner loop
Iteration 2, inertia 10818.28968253968
center shift 0.000000e+00 within tolerance 4.622175e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 11214.333333333334
start iteration


## Feature selection using Random Forest

### Feature Selection 

### Get the inportant features from random forest

In [354]:
if enable_rf_features:
    print("Random Forest Feature Selection: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    forest = RandomForestClassifier(n_estimators=250, random_state=0, n_jobs=-1, verbose=verbose_level)
    print("Random Forest Feature Selection: Fit Start")
    forest.fit(X, Y)
    print("Random Forest Feature Selection: Fit")

    importFeatures = forest.feature_importances_
    print("Random Forest Feature Selection: Feature Importance")
    print(importFeatures)
    
    indices = np.argsort(importFeatures)[::-1]
    print(indices)
    featureLabel = X.columns[0:]
    print(featureLabel)
    rankedFeature = []
    for f in range(X.shape[1]):
        rankedFeature.append(featureLabel[indices[f]])
        print("%2d) %-*s %f" % (f+1, 30,  featureLabel[indices[f]], importFeatures[indices[f]]))
    print(rankedFeature)

### Reduce the number of features

In [355]:
#select features that contribute more than 0.05
if enable_rf_features:
    X_Selected = X[rankedFeature[0:10]]
else:
    if feature_all:
        X_Selected = X
    else:
        X_Selected = X[defaultFeatures]

if pyscript:
    print(X_Selected)
else:
    display(X_Selected)

t_end =  time.time()
print(time.asctime( time.localtime(t_end) ))

print("Random Forest Feature Selection: End")


        C_YEAR C_MNTH C_WDAY C_HOUR C_VEHS C_CONF C_RCFG C_WTHR C_RSUR C_RALN  \
1497736   2005      7      5     16      2     21      2      1      1      2   
3059749   2013      8      5      8      2     36      3      1      1      1   
1908179   2007      6      6     21      3     31      2      1      1      1   
737129    2002      3      4     18      2     31      2      1      1      1   
345811    2000      7      4     13      2     31      2      1      1      1   
3372130   2015      6      2     14      3     33      2      1      1      1   
2647942   2011      5      5     18      2     35      3      1      1      1   
1760000   2006     10      1     23      1      6      1      4      4      1   
1321151   2004      9      6     13      2     31      1      2      2      2   
942120    2003      1      4     14      3     21      1      1      1      3   
3425156   2015      9      5     10      2     35      2      1      1      1   
1308966   2004      9      2

In [356]:
print("Split Test and Train based on Selected Features")
#sprint into train and test 70/30
X_train, X_test, Y_train, Y_test = train_test_split(X_Selected, Y, test_size=0.3, random_state=0)

Split Test and Train based on Selected Features


### SVM GridSearch for Optimal Parms

In [357]:
#This operation is computationaly expensive.
if enable_grid_search:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    param_grid = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose=verbose_level)
    grid.fit(X_train, Y_train)
    print(grid.best_params_)
    svm_c = grid.best_params_.get('C')
    svm_gamma = grid.best_params_.get('gamma')
    print(grid.best_estimator_)
    grid_predictions = grid.predict(X_test)
    cfn_matrix_grid = confusion_matrix(Y_test, grid_predictions)
    print(cfn_matrix_grid)
    print(classification_report(Y_test,grid_predictions))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

Sun Nov 11 11:30:33 2018
Fitting 3 folds for each of 25 candidates, totalling 75 fits
{'C': 1, 'gamma': 0.001}
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
[[2 0]
 [1 3]]
             precision    recall  f1-score   support

          0       0.67      1.00      0.80         2
          1       1.00      0.75      0.86         4

avg / total       0.89      0.83      0.84         6

Sun Nov 11 11:30:33 2018


[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.4s finished


## Logistic Regression Model

In [358]:
if enable_logistic_regression:
    print("Logistic Regression: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(C=1, random_state=0, verbose=verbose_level)
    print("Logistic Regression: Fit")
    lr.fit(X_train, Y_train)
    print("Logistic Regression: Predict")
    y_pred = lr.predict(X_test)

    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    # print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Intercept")
    print(lr.intercept_)

    # print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Coefficients")
    print(lr.coef_)

    print("Logistic Regression: Confusion Matrix")
    cnf_matrix_lg = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg)
    
    print("Logistic Regression: Classification Report")
    print(classification_report(Y_test, y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Logistic Regression: End")

Logistic Regression: Start
Sun Nov 11 11:30:33 2018
Logistic Regression: Fit
[LibLinear]Logistic Regression: Predict
Accuracy of logistic regression classifier on train set: 1.00
Accuracy of logistic regression classifier on test set: 0.83
Logistic Regression: Intercept
[0.00092619]
Logistic Regression: Coefficients
[[ 0.2639404  -0.49979646 -0.36431306  0.47560835 -0.68541925 -0.01019012
   0.42673736  0.31623717  0.54542818  0.08781967  0.22732972 -0.31462104
  -0.05327987 -0.2753      0.35185206  0.21527758  0.65930677  0.0316277
   0.00185237  0.27041625]]
Logistic Regression: Confusion Matrix
[[2 0]
 [1 3]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.67      1.00      0.80         2
          1       1.00      0.75      0.86         4

avg / total       0.89      0.83      0.84         6

Sun Nov 11 11:30:33 2018
Logistic Regression: End


### Logistic Regression with L1 Regularization

In [359]:
if enable_logistic_regression_l1:
    # with L1 regularization
    print("Logistic Regression with L1 Regularization: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(penalty='l1', C=1, random_state=0, verbose=verbose_level)
    print("Logistic Regression with L1 Regularization: Fit")
    lr.fit(X_train, Y_train)
    
    print("Logistic Regression with L1 Regularization: Predict")
    y_pred = lr.predict(X_test)
    
    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    print("Logistic Regression with L1 Regularization: Confusion Matrix")
    cnf_matrix_lg_l1 = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg_l1)
    
    print(classification_report(Y_test,y_pred))
    print("Logistic Regression with L1 Regularization: Classification Report")

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Logistic Regression with L1 Regularization: End")

Logistic Regression with L1 Regularization: Start
Sun Nov 11 11:30:34 2018
Logistic Regression with L1 Regularization: Fit
[LibLinear]Logistic Regression with L1 Regularization: Predict
Accuracy of logistic regression classifier on train set: 0.93
Accuracy of logistic regression classifier on test set: 0.83
Logistic Regression with L1 Regularization: Confusion Matrix
[[2 0]
 [1 3]]
             precision    recall  f1-score   support

          0       0.67      1.00      0.80         2
          1       1.00      0.75      0.86         4

avg / total       0.89      0.83      0.84         6

Logistic Regression with L1 Regularization: Classification Report
Sun Nov 11 11:30:34 2018
Logistic Regression with L1 Regularization: End


C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\svm\base.py:898: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Decision Tree

In [360]:
if enable_decision_tree:
    print("Decision Tree: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    tree = DecisionTreeClassifier(criterion='entropy',max_depth=100, random_state=0)
    print("Decision Tree: Fit")
    tree.fit(X_train, Y_train)
    
    print("Decision Tree: Predict")
    y_pred = tree.predict(X_test)
    print('Accuracy of Decision Tree classifier on train set: {:.2f}'.format(tree.score(X_train, Y_train)))
    print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(tree.score(X_test, Y_test)))
    
    cnf_matrix_dt = confusion_matrix(Y_test, y_pred)
    print("Decision Tree: Confusion Matrix")
    print(cnf_matrix_dt)
    print("Decision Tree: Classification Report")
    print(classification_report(Y_test,y_pred))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    print("Decision Tree: End")

Decision Tree: Start
Sun Nov 11 11:30:34 2018
Decision Tree: Fit
Decision Tree: Predict
Accuracy of Decision Tree classifier on train set: 1.00
Accuracy of Decision Tree classifier on test set: 0.83
Decision Tree: Confusion Matrix
[[2 0]
 [1 3]]
Decision Tree: Classification Report
             precision    recall  f1-score   support

          0       0.67      1.00      0.80         2
          1       1.00      0.75      0.86         4

avg / total       0.89      0.83      0.84         6

Sun Nov 11 11:30:34 2018
Decision Tree: End


### K-N-N

In [361]:
if enable_knn:
    print("KNN: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
    print("KNN: Fit")
    knn.fit(X_train, Y_train)
    
    print("KNN: Predict")
    y_pred = knn.predict(X_test)
    print('Accuracy of KNN classifier on train set: {:.2f}'.format(knn.score(X_train, Y_train)))
    print('Accuracy of KNN classifier on test set: {:.2f}'.format(knn.score(X_test, Y_test)))
    
    print("KNN: Confusion Matrix")
    cnf_matrix_knn = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_knn)

    print("KNN: Classification Report")
    print(classification_report(Y_test,y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

    print("KNN: End")

KNN: Start
Sun Nov 11 11:30:34 2018
KNN: Fit
KNN: Predict
Accuracy of KNN classifier on train set: 0.57
Accuracy of KNN classifier on test set: 0.67
KNN: Confusion Matrix
[[1 1]
 [1 3]]
KNN: Classification Report
             precision    recall  f1-score   support

          0       0.50      0.50      0.50         2
          1       0.75      0.75      0.75         4

avg / total       0.67      0.67      0.67         6

Sun Nov 11 11:30:34 2018
KNN: End


### SVM

In [362]:
if enable_svm:
    print("SVM: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    #svm = SVC(C=1, random_state=0, kernel='sigmoid', verbose=True)
    #svm = SVC(C=1, random_state=0, kernel='linear', verbose=True, cache_size=200)
    svm = SVC(C=svm_c, gamma=svm_gamma, verbose = verbose_level)
    print("SVM: Fit")
    svm.fit(X_train, Y_train)
    
    print("SVM: Predict")
    y_pred = svm.predict(X_test)
    
    print('Accuracy of SVM classifier on train set: {:.2f}'.format(svm.score(X_train, Y_train)))
    print('Accuracy of SVM classifier on test set: {:.2f}'.format(svm.score(X_test, Y_test)))
    
    print("SVM: Confusion Matrix")
    cnf_matrix_svm = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_svm)
    
    print("SVM: Classfication Report")
    print(classification_report(Y_test,y_pred))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("SVM: End")

SVM: Start
Sun Nov 11 11:30:34 2018
SVM: Fit
[LibSVM]SVM: Predict
Accuracy of SVM classifier on train set: 0.71
Accuracy of SVM classifier on test set: 0.83
SVM: Confusion Matrix
[[2 0]
 [1 3]]
SVM: Classfication Report
             precision    recall  f1-score   support

          0       0.67      1.00      0.80         2
          1       1.00      0.75      0.86         4

avg / total       0.89      0.83      0.84         6

Sun Nov 11 11:30:34 2018
SVM: End


## ANN - Multilayer Perceptron

In [363]:
if enable_mlp:
    print("Multilayer Preceptron: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    #mlpc = MLPClassifier(alpha=1)
    mlpc = MLPClassifier(hidden_layer_sizes=(12, 12, 12), max_iter=1000, verbose=verbose_level)
    
    #mlp = multilayer_perceptron(n_hidden =2, activation='logistic', algorithm='sgd', random_state=3)
    print("Multilayer Preceptron: fit")
    mlpc.fit(X_train, Y_train)
    
    print("Multilayer Preceptron: Predict")
    y_pred = mlpc.predict(X_test)

    print('Accuracy of Multilayer Perceptron classifier on train set: {:.2f}'.format(mlpc.score(X_train, Y_train)))
    print('Accuracy of Multilayer Perceptron classifier on test set: {:.2f}'.format(mlpc.score(X_test, Y_test)))
    
    print("Multilayer Preceptron: Confusion Matrix")
    cnf_matrix_mlp = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_mlp)
    
    print("Multilayer Preceptron: Classificiation Report")
    print(classification_report(Y_test,y_pred))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Multilayer Preceptron: End")

Multilayer Preceptron: Start
Sun Nov 11 11:30:34 2018
Multilayer Preceptron: fit
Iteration 1, loss = 9.50233944
Iteration 2, loss = 7.68951398
Iteration 3, loss = 5.73477006
Iteration 4, loss = 3.79773224
Iteration 5, loss = 1.91898783
Iteration 6, loss = 0.67702577
Iteration 7, loss = 1.80239473
Iteration 8, loss = 2.79859030
Iteration 9, loss = 3.06112435
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Multilayer Preceptron: Predict
Accuracy of Multilayer Perceptron classifier on train set: 0.43
Accuracy of Multilayer Perceptron classifier on test set: 0.67
Multilayer Preceptron: Confusion Matrix
[[0 2]
 [0 4]]
Multilayer Preceptron: Classificiation Report
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         2
          1       0.67      1.00      0.80         4

avg / total       0.44      0.67      0.53         6

Sun Nov 11 11:30:34 2018
Multilayer Preceptron: End


C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Random forest

In [364]:
if enable_rf:
    print("Ensemble (Bagging): Random Forest: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    forest = RandomForestClassifier(criterion='entropy', n_estimators=250, random_state=0, n_jobs=2, verbose=verbose_level)
    print("Ensemble (Bagging): Random Forest: Fit")
    forest.fit(X_train, Y_train)
    
    print("Ensemble (Bagging): Random Forest: Predict")
    y_pred = forest.predict(X_test)
    
    print('Accuracy of RandomForest classifier on train set: {:.2f}'.format(forest.score(X_train, Y_train)))
    print('Accuracy of RandomForest classifier on test set: {:.2f}'.format(forest.score(X_test, Y_test)))
    
    print("Ensemble (Bagging): Random Forest: Confusion Matrix")
    cnf_matrix_rf = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_rf)
    
    print("Ensemble (Bagging): Random Forest: Classification Report")
    print(classification_report(Y_test,y_pred))

    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    print("Ensemble (Bagging): Random Forest: End")

Ensemble (Bagging): Random Forest: Start
Sun Nov 11 11:30:34 2018
Ensemble (Bagging): Random Forest: Fit


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s


Ensemble (Bagging): Random Forest: Predict


[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.0s finished


Accuracy of RandomForest classifier on train set: 1.00
Accuracy of RandomForest classifier on test set: 0.67
Ensemble (Bagging): Random Forest: Confusion Matrix
[[2 0]
 [2 2]]
Ensemble (Bagging): Random Forest: Classification Report
             precision    recall  f1-score   support

          0       0.50      1.00      0.67         2
          1       1.00      0.50      0.67         4

avg / total       0.83      0.67      0.67         6

Sun Nov 11 11:30:34 2018
Ensemble (Bagging): Random Forest: End


In [365]:

#check sigmoid and rbf
#from sklearn.ensemble import BaggingClassifier
#from sklearn.svm import SVC
#clf = BaggingClassifier(SVC(C=1.0,
#        cache_size=200,
#        class_weight=None,
#        coef0=0.0,
#        decision_function_shape=None,
#        degree=3,
#        gamma='auto',
#        kernel='linear',
#        max_iter=-1,
#        probability=False,
#        random_state=None,
#        shrinking=True,
#        tol=0.001,
#        verbose=False,
#        ))

In [366]:
#clf.fit(X_train, Y_train)

In [367]:
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ))

Sun Nov 11 11:30:34 2018


In [368]:
#sys.stdout.close()